# Voltage Imaging Analysis Benchmark

## For Empirical Code Evaluation

This notebook defines a benchmark for automated optimization of voltage imaging analysis pipelines, suitable for empirical code evaluation frameworks that iteratively search for high-performing code variants.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/voltage-imaging-benchmark/blob/main/voltage_imaging_benchmark.ipynb)

---
# 0. Setup and Installation


In [1]:
# Install dependencies (run this cell in Colab or if packages are missing)
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install -q numpy scipy matplotlib scikit-image scikit-learn pandas seaborn tifffile nd2 opencv-python gdown umap-learn hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.5/234.5 kB 16.7 MB/s eta 0:00:00


## 0.2 Download Benchmark Data

The benchmark data is hosted on Google Drive. Run the cell below to download it.

In [ ]:
import os
from pathlib import Path

# === CONFIGURE YOUR GOOGLE DRIVE FILE ID HERE ===
# To get the file ID from a Google Drive sharing link:
# https://drive.google.com/file/d/FILE_ID_HERE/view?usp=sharing
#                                  ^^^^^^^^^^^^ copy this part

GDRIVE_FILE_ID = "156ASrvQfbyjrHwtAeCCt_dTuWcktS9Cu"  # Replace with your actual file ID
DATA_DIR = Path("./data")
DATA_DIR.mkdir(exist_ok=True)

def download_data(file_id, output_dir):
    """Download and extract benchmark data from Google Drive."""
    import gdown
    import zipfile

    zip_path = output_dir / "benchmark_data.zip"

    # Check if data already exists
    if (output_dir / "video.tif").exists() or (output_dir / "fish1").exists():
        print("Data already downloaded. Skipping.")
        return

    # Download from Google Drive
    url = f"https://drive.google.com/uc?id={file_id}"
    print(f"Downloading data from Google Drive...")
    gdown.download(url, str(zip_path), quiet=False)

    # Extract if it's a zip file
    if zipfile.is_zipfile(zip_path):
        print("Extracting data...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
        zip_path.unlink()  # Remove zip after extraction
        print("Done!")
    else:
        # Might be a single TIFF file
        print("Downloaded file (not a zip archive)")

# Uncomment to download:
# download_data(GDRIVE_FILE_ID, DATA_DIR)

## 0.3 Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage, signal
from skimage import measure, morphology
import tifffile
from pathlib import Path
import json
from typing import List, Dict, Tuple, Optional

# Set plotting style
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("Libraries imported successfully!")

---
# 1. Problem Definition

## 1.1 Scientific Context

**Voltage imaging** enables recording of neuronal electrical activity at high temporal resolution (~200 Hz) across many neurons simultaneously. Unlike calcium imaging, voltage indicators directly report membrane potential changes, capturing:
- **Action potentials (spikes)** - fast (~1-5ms) electrical events
- **Subthreshold activity** - slower membrane potential fluctuations

## 1.2 The Data

| Property | Value |
|----------|-------|
| **Organism** | Zebrafish (larval) |
| **Indicator** | Voltron-2 (soma-targeted) |
| **Frame rate** | ~200 Hz (5 ms/frame) |
| **File format** | ND2 (Nikon) → TIFF |
| **File size** | 2-10 GB per recording |
| **Duration** | 1,000-10,000 frames (5-50 seconds) |
| **Neurons per FOV** | 10-100 |
| **Neuron size** | ~5-10 µm (~9 pixels) |
| **Neuron appearance** | Ring-like (membrane labeling) |

## 1.3 Signal Characteristics

**Important**: Voltron-2 produces **NEGATIVE deflections** during action potentials.

| Feature | Typical Value |
|---------|---------------|
| Spike duration | 1-5 ms (1-2 frames at 200 Hz) |
| Spike amplitude | 2-10% ΔF/F |
| Spike polarity | **Negative** (downward) |
| Noise type | Broadband (camera/shot noise) |
| SNR | Variable, typically 2-10 |

---
# 2. Task Specification

## 2.1 Overall Goal

Given a raw voltage imaging video, produce:
1. **ROI masks** - binary masks identifying each neuron
2. **Spike times** - timestamps of detected action potentials for each neuron
3. **Voltage traces** - cleaned ΔF/F time series for each neuron

## 2.2 Pipeline Components (Optimization Targets)

The analysis pipeline consists of sequential processing stages. Each stage can be independently optimized:

```
┌─────────────────────────────────────────────────────────────────────┐
│                    OPTIMIZATION TARGETS                             │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  STAGE 1: MOTION CORRECTION                                         │
│  ├── Method: rigid vs non-rigid                                    │
│  ├── Reference: first frame, mean, median, specific frame          │
│  ├── Algorithm: phase correlation, template matching, optical flow │
│  └── Parameters: max_shift, upsample_factor, smoothing             │
│                                                                     │
│  STAGE 2: DENOISING                                                 │
│  ├── Method: none, temporal filter, spatial filter, PCA, wavelet   │
│  ├── Temporal: lowpass cutoff, Savitzky-Golay window, median       │
│  ├── Spatial: Gaussian sigma, bilateral filter                     │
│  └── PCA: local vs global, number of components, patch size        │
│                                                                     │
│  STAGE 3: ROI SEGMENTATION                                          │
│  ├── Method: threshold, watershed, CNN (Mask R-CNN), NMF           │
│  ├── Features: std projection, correlation image, PCA components   │
│  ├── Constraints: min/max area, circularity, ring-like shape       │
│  └── Post-processing: merge overlapping, remove duplicates         │
│                                                                     │
│  STAGE 4: TRACE EXTRACTION                                          │
│  ├── Aggregation: mean, median, weighted by distance               │
│  ├── Background: none, annulus subtraction, neuropil coefficient   │
│  ├── Baseline: percentile (which?), rolling window size            │
│  └── Detrending: none, linear, polynomial, exponential             │
│                                                                     │
│  STAGE 5: SPIKE DETECTION                                           │
│  ├── Method: threshold, template matching, deconvolution, ML       │
│  ├── Threshold: fixed std, adaptive, percentile-based              │
│  ├── Constraints: min spike width, refractory period               │
│  └── Post-processing: amplitude filter, artifact rejection         │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

## 2.3 Input Format

```python
# Input: 3D numpy array
video: np.ndarray  # shape: (n_frames, height, width), dtype: uint16 or float32
fps: float  # frame rate in Hz (typically 200)
```


---
# 3. Challenges and Failure Modes

## 3.1 Motion Artifacts

**Problem**: The animal moves during imaging, causing:
- Rigid translation (x/y shifts)
- Non-rigid deformation (tissue stretching)
- Focus changes (neurons going in/out of focus)
- ROI contamination (wrong pixels included after movement)

**Failure modes**:
- Correlated "spikes" across many neurons (motion artifact detected as spike)
- Lost neurons (moved out of ROI)
- False spikes from intensity changes due to focus shifts

## 3.2 Segmentation Challenges

**Problem**: Neurons have irregular shapes:
- Ring-like appearance (soma-targeted membrane labeling)
- Variable brightness
- Overlapping neurons
- Similar size to noise blobs

**Failure modes**:
- Merging adjacent neurons into one ROI
- Splitting one neuron into multiple ROIs
- Including non-neuronal structures
- Missing dim neurons

## 3.3 Spike Detection Challenges

**Problem**: Spikes are fast and small:
- Only 1-2 frames wide at 200 Hz
- Low SNR (amplitude similar to noise)
- Variable amplitude across neurons and within same neuron
- Negative polarity (opposite to calcium indicators)

**Failure modes**:
- Missing true spikes (false negatives)
- Detecting noise as spikes (false positives)
- Correlated false detections from global artifacts

## 3.4 Known Artifacts

| Artifact | Cause | Detection Method |
|----------|-------|------------------|
| Photobleaching | Dye degradation | Slow intensity decay |
| Focus drift | Mechanical drift | Blur changes over time |

---
# 4. Evaluation Metrics

## 4.1 Ground Truth Comparison (Supervised)

When expert annotations are available

## 4.2 Automated Validation (Unsupervised)

Metrics that don't require ground truth annotations:

### 4.2.1 Shuffled Control Test

**Principle**: Spike detection on temporally shuffled data should yield far fewer spikes than real data.

**Interpretation**:
- `spike_ratio >> 1`: Good, detecting real structure
- `spike_ratio ≈ 1`: Bad, just detecting noise

### 4.2.2 Spike Template Consistency

**Principle**: Spikes from the same neuron should have similar waveform shapes.

**Interpretation**:
- `mean_correlation > 0.8`: Good, consistent spike shapes
- `mean_correlation < 0.5`: Bad, likely detecting noise

### 4.2.3 Inter-Neuron Correlation (Artifact Detection)

**Principle**: High correlation in spike timing across many neurons suggests artifacts.

**Interpretation**:
- `mean_correlation < 0.1`: Good, neurons fire independently
- `mean_correlation > 0.3`: Bad, likely motion artifacts
- `synchrony_events > 0`: Suspicious, check these timepoints

### 4.2.4 ROI Quality Metrics

if within expected range (5-10 µm)

---
# 6. Baseline Implementation

A simple baseline pipeline for comparison:

```python
def baseline_pipeline(video, fps=200):
    """
    Simple baseline voltage imaging analysis pipeline.
    
    Args:
        video: np.ndarray, shape (n_frames, height, width)
        fps: frame rate in Hz
    
    Returns:
        dict with roi_masks, traces, spike_times, spike_frames
    """
    import numpy as np
    from scipy import ndimage, signal
    from skimage import measure, morphology
    
    n_frames, height, width = video.shape
    
    # === STAGE 1: No motion correction (baseline) ===
    corrected = video
    
    # === STAGE 2: Simple denoising ===
    # Temporal Gaussian smoothing
    denoised = ndimage.gaussian_filter1d(corrected.astype(float), sigma=1, axis=0)
    
    # === STAGE 3: ROI segmentation ===
    # Use std projection to find active regions
    std_proj = np.std(denoised, axis=0)
    
    # Threshold
    threshold = np.mean(std_proj) + 1.5 * np.std(std_proj)
    binary = std_proj > threshold
    
    # Clean up
    binary = morphology.remove_small_objects(binary, min_size=30)
    binary = morphology.remove_small_holes(binary, area_threshold=30)
    
    # Label connected components
    labeled = measure.label(binary)
    regions = measure.regionprops(labeled)
    
    # Filter by size
    roi_masks = []
    for region in regions:
        if 30 < region.area < 500:  # Expected neuron size
            mask = labeled == region.label
            roi_masks.append(mask)
    
    roi_masks = np.array(roi_masks)
    
    # === STAGE 4: Trace extraction ===
    traces = []
    for mask in roi_masks:
        trace = np.mean(denoised[:, mask], axis=1)
        # Compute dF/F
        f0 = np.percentile(trace, 10)
        dff = (trace - f0) / f0
        # Detrend
        dff = signal.detrend(dff)
        traces.append(dff)
    
    traces = np.array(traces)
    
    # === STAGE 5: Spike detection ===
    spike_times = []
    spike_frames = []
    
    for trace in traces:
        # Invert (Voltron-2 has negative spikes)
        inverted = -trace
        
        # Threshold at 3 std
        threshold = np.mean(inverted) + 3 * np.std(inverted)
        
        # Find peaks
        peaks, _ = signal.find_peaks(
            inverted,
            height=threshold,
            distance=int(0.005 * fps)  # 5ms refractory
        )
        
        spike_frames.append(peaks)
        spike_times.append(peaks / fps)
    
    return {
        'roi_masks': roi_masks,
        'traces': traces,
        'spike_times': spike_times,
        'spike_frames': spike_frames
    }
```

---
# 9. References and Prior Work

## 9.1 Existing Tools

| Tool | What it does | Strengths | Limitations |
|------|--------------|-----------|-------------|
| **VolPy** (CaImAn) | Full pipeline | Mask R-CNN segmentation, SpikePursuit | Complex setup |
| **NoRMCorre** | Motion correction | Well-validated | Part of CaImAn |
| **Suite2p** | Calcium imaging | Fast, scalable | Not optimized for voltage |

## 9.2 Key Papers

1. **VolPy** - Cai et al., 2021, PLOS Comp Bio
   - Mask R-CNN for segmentation
   - SpikePursuit for spike detection
   - F1 > 90% on benchmark data

2. **Voltage imaging pipeline** - (First paper you shared)
   - Camera noise correction
   - Local PCA denoising
   - Semi-NMF segmentation
   - LSTM spike detection

3. **Whole-brain voltage imaging** - (Second paper - Positron2)
   - NoRMCorre motion correction
   - UMAP+DBSCAN artifact removal
   - SNR > 4 filtering

## 9.3 Novel Ideas to Explore

- Shuffled control validation
- Template consistency scoring
- Artifact correlation detection
- Combined supervised + unsupervised evaluation

---
# 9.5 Implementation Ideas from related papers

# 9.5.1 Voltron-2 analysis in small FOV paper

This is from the paper
Voltage imaging reveals circuit computations in the raphe underlying serotonin-mediated motor vigor learning
Takashi Kawashima1,2,4 takashi.kawashima@weizmann.ac.il ∙ Ziqiang Wei1,4 weiz@janelia.hhmi.org ∙ Ravid Haruvi2 ∙ Inbal Shainer2,3 ∙ Sujatha Narayan1 ∙ Herwig Baier3 ∙ Misha B. Ahrens
Imaging Voltron in zebrafish

Processing pipeline for voltage imaging data
We built a data processing pipeline for voltage imaging data to automatically perform camera-noise correction, motion correction, denoising, and cell segmentation (Figure 2B).
Step #1: Camera-noise correction
We adopted the camera noise correction algorithm from refs. Huang et al.90 and Liu et al.91 The pixel readout is computed as
𝑠𝑖=(𝑠𝑖𝑟−𝑜𝑖)/𝑔𝑖
where 𝑠𝑖𝑟 is the raw camera readout at pixel i, 𝑠𝑖 is the corrected one, 𝑜𝑖 is the camera offset, and 𝑔𝑖 is the camera gain at pixel i. We estimated the offset 𝑜𝑖 and baseline variance 𝑣𝑖 in the no-light condition as the mean and variance of 60k images. The gain for each pixel was calculated as
𝑔𝑖=𝑎𝑟𝑔⁡
𝑚𝑖𝑛
𝑔
 ⁡∑𝐾
𝑘=1((𝑣𝑖𝑘−𝑣𝑖)−𝑔⁢(𝐷𝑖𝑘−𝑜𝑖))2,
where K is the total number of illumination levels acquired, k is the kth illumination level, 𝐷𝑖𝑘 and 𝑣𝑖𝑘 are the mean and variance of the 20k images acquired in the kth illumination level. In our setup, we measured them by varying the laser power from 0 mW (no light condition) to 18 mW. Empirically, we found that the camera-noise correction can improve motion correction and other later steps in the processing.91,92
Step #2: Motion correction
We performed the two-dimensional rigid registration of the images using a custom Python script based on the dipy package,93 to correct drifts in the sequentially recorded images at the subpixel level.
Step #3: image-series denoising
We performed image-series denoising for motion-corrected video Y by finding its low-rank (at K) representation
̅
𝑌𝑘
=∑𝐾
𝑘=1𝑢𝑘⁢𝑣𝑘
with residual 𝑅𝑘=𝑌−
̅
𝑌𝑘
 is statistically white (within 99% confidence interval). Here, 𝑢𝑘, 𝑣𝑘 are spatial and temporal components respectively and determined by the objective
𝑢𝑘,𝑣𝑘=𝑎𝑟𝑔⁡
𝑚𝑖𝑛
𝑢,𝑣
 ⁡‖𝑅𝑘−1⁢−𝑢⁢𝑣‖𝐹2
where F denotes the Frobenius norm. This is equivalent to performing iterative principal components analysis (PCA) on Y and stopping the iteration at the kth component as the residual is close to the white noise. Since the number of pixels is large, which creates difficulty in computing PCA, we alternatively divided the whole image into overlapped small patches (namely local PCA), performed this denoising procedure for each patch, and stitched the denoised patches back to a full denoised movie. Moreover, since video Y can be presented as a 3D tensor, we also tested the tensor-based optimization as
̅
𝑌𝑘
=∑𝐾
𝑘=1𝑢1,𝑘∘𝑢2,𝑘∘𝑣𝑘
where 𝑢1,𝑘 and 𝑢2,𝑘 are two spatial components along the vertical and horizontal directions of the imaging, respectively. We found that tensor decomposition would generate stripe-like spatial correlation horizontally (which might stem from the power fluctuation of the horizontal light-sheet illumination), and thus used local PCA instead throughout the denoising step.
Step #4: Cell segmentation
We performed cell segmentation on the denoised movie (
̅
𝑌𝑘
 ) using semi-nonnegative factorization, where
𝐴,𝐹,𝐵=𝑎𝑟𝑔⁡
𝑚𝑖𝑛
𝐴,𝐹,𝐵
 ⁡‖
̅
𝑌𝑘
⁢−𝐴⁢𝐹−𝐵‖𝐹2
such that
𝐴≥0,𝐵=𝑏·1𝑇,𝑏≥0
where A is the matrix of the nonnegative components in which the pixels were connected in space (ROIs); F is the matrix of the temporal components and presents the fluorescent dynamics of ROIs, and B is the temporally constant background. We initialized the components using super-pixels,94 which include the local pixels with neighboring correlations larger than 0.8. We then computed
Δ⁢𝐹/𝐹=
𝐹−𝐹0
𝐹0

for each component (or ROI), where baseline fluorescence 𝐹0 is computed as a running 20% percentile of F within a 3-minute time window. In our hands, the denoising step does not affect the dynamics of spike generation (thus spike detection) or membrane potential fluctuation, compared to the computation of ΔF/F using hand-drawn ROIs. Since Voltron is a negative response indicator, we used the flipped signal (−ΔF/F) for subsequent analyses for spike detection and subthreshold activity estimation.
Step #5: Spike detection
We trained a machine-learning neural network (Figure 2C) in order to automatically extract the spikes from the time series data of ΔF/F. The network consists of two LSTM layers and a dropout layer that interconnects them. To train this neural network, we used simultaneous electrophysiology and imaging of cerebellar neurons expressing Voltron acquired in our previous work.26 The goal of the network is to determine whether there is a spike event in the middle of the time series (the length of the time series is 41 frames or 136.67 ms) by minimizing the loss function of the cross-entropy between the prediction of spike from voltage imaging with the ground truth from the electrophysiology (Figure 2C; gray dots, ground truth from electrophysiology; black, prediction of the network). We performed the 10-fold cross-validation in network training. We applied the trained network model to extract the spikes automatically thereafter.
Step #6: Subthreshold activity estimation
We estimated the subthreshold activity as a rolling median filter of time series data of ΔF/F with a window size of 70 ms. To avoid the nonlinearity of spikes (a depolarization followed by a repolarization), we clipped out the frames (from -1 to +1) around detected spikes while running the median filter.


Kernel fits of membrane potential and spikes to behavioral variables
Response kernel estimation of serotonergic neurons based on behavioral variables (Figure 3E) was performed as follows. We predicted spike events and subthreshold membrane potentials from the behavioral variables of the time series of the swim vigor
𝑆𝑡={𝑠𝑡,𝑠𝑡−1,𝑠𝑡−2,⋯,𝑠𝑡−𝜏}
and that of the visual input
𝑉𝑡={𝑣𝑡,𝑣𝑡−1,𝑣𝑡−2,⋯,𝑣𝑡−𝜏}
and that of the recent spike events 𝑆⁢𝑃𝑡={𝑠⁢𝑝𝑡,𝑠⁢𝑝𝑡−1,𝑠⁢𝑝𝑡−2,⋯,𝑠⁢𝑝𝑡−𝜏}, where 𝑠⁢𝑝 is 0 or 1,at a given time t, allowing history dependence (t- τ) up to 300 time points (1 second). Since spiking activity is sampled at 300 Hz, for each single frame, the spike event is binary, we thus modeled the spike event as
𝑃⁡(𝑠⁢𝑝⁢𝑖⁢𝑘⁢𝑒⁢𝑠𝑎⁢𝑡𝑡⁢𝑖⁢𝑚⁢𝑒𝑡)=𝐵⁢𝑖⁢𝑛⁢𝑜⁢𝑚⁢𝑖⁢𝑎⁢𝑙⁢(𝑤𝑠𝑇⁢𝑆𝑡+𝑤𝑣𝑇⁢𝑉𝑡−𝑤𝑠⁢𝑝𝑇⁢𝑆⁢𝑃𝑡).
We optimized this probabilistic model from the data using a generalized linear model with logit link function and a 𝑙2-form regularization on kernels of the swim vigor, i.e. 𝑤𝑠, the visual input, i.e. 𝑤𝑣, and the spike history, i.e. 𝑤𝑠⁢𝑝. The model was fitted in Python using 𝑙2 logistic regression in the Scikit-learn package95 with the following steps for data collection and cross-validation. The time series data was sampled randomly around spike events, with a balance of 𝑃⁡(𝑠⁢𝑝⁢𝑖⁢𝑘⁢𝑒⁢𝑠𝑎⁢𝑡𝑡⁢𝑖⁢𝑚⁢𝑒𝑡)≃0.5. We determined the strength of 𝑙2-form regularization using 5-fold cross-validation. In each fold, we held 20% of the data for validation (completely non-overlapped with the rest of the data in the original full time series) and used the remaining 80% of the data for training. Moreover, we used the square root form of the swim vigor data in fitting. We used a model without regularization on validation data as the full model and determined the performance, using negative log likelihood (NLL), as
1−
𝑁⁢𝐿⁡𝐿⁡(𝑃𝑒⁢𝑠⁢𝑡⁡(𝑠⁢𝑝⁢𝑖⁢𝑘⁢𝑒⁢𝑠))
𝑁⁢𝐿⁡𝐿⁡(𝑃𝑡⁢𝑟⁢𝑢⁢𝑒⁡(𝑠⁢𝑝⁢𝑖⁢𝑘⁢𝑒⁢𝑠))

.
This is equivalent to a measure of the explained variance in a logistic regression.
Furthermore, we assumed the change of subthreshold membrane potentials linearly as
Δ⁢𝐹⁡/⁢𝐹⁡(𝑡)−Δ⁢𝐹⁡/⁢𝐹⁡(𝑡−𝜏)=𝑘⁢(𝑤𝑠𝑇⁢𝑆𝑡+𝑤𝑣𝑇⁢𝑉𝑡−𝑤𝑠⁢𝑝𝑇⁢𝑆⁢𝑃𝑡)+𝑐,
and thus the model error is
𝑒⁡(Δ⁢𝐹⁡/⁢𝐹⁡(𝑡))=Δ⁢𝐹⁡/⁢𝐹⁡(𝑡)−Δ⁢𝐹⁡/⁢𝐹⁡(𝑡−𝜏)−𝑘⁢(𝑤𝑠𝑇⁢𝑆𝑡+𝑤𝑣𝑇⁢𝑉𝑡−𝑤𝑠⁢𝑝𝑇⁢𝑆⁢𝑃𝑡)+𝑐,
where k and c are two scalars in model fits.
We then fit the combined model of spike events and the change of subthreshold membrane potential as
𝑤𝑠,𝑤𝑣,𝑤𝑠⁢𝑝=𝑎𝑟𝑔⁡
𝑚𝑖𝑛
𝑤𝑠,𝑤𝑣,𝑤𝑠⁢𝑝
 ⁡𝑁⁢𝐿⁡𝐿⁡(𝑃⁡(𝑠⁢𝑝⁢𝑖⁢𝑘⁢𝑒⁢𝑠))+
1
2

⁢||𝑒⁢(Δ⁢𝐹/𝐹)|∣22+
1
2

⁢𝛼⁡(‖𝑤𝑠‖22⁢+‖⁢𝑤𝑣⁢‖22+‖⁢𝑤𝑠⁢𝑝∥22)
where 𝛼 is 𝑙2-form regularization of the kernels. We performed gradient descent-based minimization with the initialization of the parameters fitted from the spike-event-only model above.
Cell selections
Figure 3C included all segmented cells from gain adaptation tasks (Figure 1B) and memory tasks (Figure S1D; only using them in low and high Gms) and ablation tasks (Figure 6; only using those before ablation) (we also fitted them all with the behavioral variables in Figure 3E). In Figure 3C, we sorted neurons based on their center of mass of spiking rates after swimming onset and separated them into high- or low-Gms preferred categories.
Neural population codes
We computed the population coding of motor vigor learning by serotonergic neurons (Figures 6F and S8F) using a sparse version of linear discriminant analysis:96
𝑙=𝑎⁢𝑟⁢𝑔⁢
𝑚𝑖𝑛
𝑙
 −
(𝑙𝑇⁢(𝑟𝑡ℎ⁢𝑖⁢𝑔⁢ℎ−𝑟𝑡𝑙⁢𝑜⁢𝑤))2
𝑙𝑇⁢𝛴𝑟⁢𝑙

.
Here 𝑟𝑡 is 𝑛-dim vector of the neural activity at time t; 𝑛 is the number of neurons in the population codes; 𝑟𝑡ℎ⁢𝑖⁢𝑔⁢ℎ and 𝑟𝑡𝑙⁢𝑜⁢𝑤 are the sample average of 𝑟𝑡 at high or low Gms conditions, respectively; 𝑙 is the coding direction for high Gms. We optimize the coding direction 𝑙 with a 𝑙2 regularizer, 𝛾∈[0,1], which was applied to the covariance matrix of the sample data
𝛴𝑟=(1−𝛾)⁢(𝑟𝑡−<𝑟𝑡>)⁢(𝑟𝑡−<𝑟𝑡>)𝑇+𝛾⁢𝐼.
We sampled the data from the late phase of each Gms period. The sample window for each data point to compute the firing rate is 1 second.
For the analysis of the neural population codes for Gms in high-speed calcium imaging (Figure S5G), we used all identified neurons in each area (Figure S5H) and performed the following procedure. We first applied a Gaussian smoothing filter (σ = 3.3 ms) to all neural traces and then subtracted the baseline ΔF/F before the swimming (100 ms) from the ΔF/F values after the swimming in individual neurons for individual swim events. We then fitted a linear decoder by regressing the ΔF/F levels 500 ms after the swims, when the neural response sufficiently diverged, in all neurons to the ratio of Gms values to that at low Gms (1 = low, 2 = medium, 3 = high Gms) in individual swim events. The resulting linear decoder was then applied to the population activity (ΔF/F values of all neurons) at individual time points around the onset of swims, and the first time point that showed a significant difference of this population vector between low and high Gms conditions across swim events (p < 0.01) are designated as the response time.
Spiking neural network model
We simulated the neural network dynamics in the dorsal raphe nucleus (Figure 5) by using a spiking neural network model. Our model was equipped with 400 serotonergic neurons and 100 GABAergic neurons with sparse random connections with probability 0.03, i.e. 𝑤𝑗⁢𝑘=1 if pc < 0.03 and otherwise 𝑤𝑗⁢𝑘=0 (where pc is sampled from a uniform distribution from 0 to 1). The serotonergic neurons are modeled as adaptive integrate-and-fire neurons97:
𝐶𝑚⁢𝑑⁢𝑉/𝑑⁢𝑡=−𝑔𝐿⁢(𝑉−𝐸𝐿)−𝑢+𝐼
𝜏𝑢⁢𝑑⁢𝑢/𝑑⁢𝑡=−𝑎⁢(𝑉−𝐸𝐿)−𝑢
V is the membrane potential; u is the adaptation variable; I is the input current; 𝐶𝑚=0.5 nF is the membrane capacitance; 𝑔𝐿=0.025 uS is the leak conductance; 𝐸𝐿=−60 mV is the leak reversal potential; 𝑎=0.01 uS is the adaptation coupling parameter (presenting the factor of the excitatory rebound currents) and 𝜏𝑢=200 ms is the adaptation time constant for rebound currents). A spike happens when 𝑉>𝑉𝑡⁢ℎ=−40 mV, and V is then reset to -50 mV for a 2-ms refractory period; at the same time, a spike triggers an inhibitory afterhyperpolarization, which would add onto the adaptation variable,
𝑢⁡(𝑡𝑠⁢𝑝⁢𝑖⁢𝑘⁢𝑒)=𝑢+𝑏,
and b = 0.001 nA. GABAergic neurons are modeled as simple integrate-and-fire neurons:
𝐶𝑚⁢𝑑⁢𝑉/𝑑⁢𝑡=−𝑔𝐿⁢(𝑉−𝐸𝐿)+𝐼
with𝐶𝑚=0.2nF;𝑔𝐿=0.02𝜇⁢S;𝐸𝐿=−60mV.
The synaptic inputs are modeled as
𝑑⁢𝑠/𝑑⁢𝑡=−
1
𝜏𝑠

⁢𝑠+∑𝑖𝛿⁢(𝑡−𝑡𝑖)
𝐼𝑠⁢𝑦⁢𝑛=𝑔𝑠⁢𝑦⁢𝑛⁢𝑠⁢(𝑉−𝑉𝑠⁢𝑦⁢𝑛)
where 𝑔𝑠⁢𝑦⁢𝑛 is a synaptic conductance, 𝑉𝑠⁢𝑦⁢𝑛 is the synaptic reversal potential, and s is a synaptic gating variable (that increases by one at a presynaptic spike event, then decays at a time constant 𝜏𝑠). We set 𝜏𝑠=50 ms; 𝑔5−𝐻⁢𝑇=0.13 ns; 𝑉5−𝐻⁢𝑇=0 mV; 𝑔𝐺⁢𝐴⁢𝐵⁢𝐴=0.52 ns; 𝑉𝐺⁢𝐴⁢𝐵⁢𝐴=−70 mV.
We provided glutamatic input into serotonergic and GABAergic neurons during the motor vigor learning task based on the results of neurotransmitter imaging. Both swim vigor and visual input were modeled as exponential decays from the behavioral variables' onset times:
𝐼𝐵⁡(𝑡)=𝐼𝐵0⁡𝑒𝑥𝑝⁡(−
𝑡−𝑡𝐵
𝜏𝐵

)
where the decay time constants were set to 50 ms, max input 𝐼𝐵0=100 pA for swim vigor, 𝐼𝐵0=50 pA for visual input. The total input to a kth serotonergic neuron was thus
𝐼𝑘=∑𝑗∈5−𝐻⁢𝑇𝑤𝑗⁢𝑘⁢𝐼𝑗⁢𝑘
5−𝐻⁢𝑇+∑𝑗∈𝐺⁢𝐴⁢𝐵⁢𝐴𝑤𝑗⁢𝑘⁢𝐼𝑗⁢𝑘
𝐺⁢𝐴⁢𝐵⁢𝐴+𝐼𝑣⁢𝑖⁢𝑠.
The total input to a kth GABAergic neuron was thus
𝐼𝑘=∑𝑗∈5−𝐻⁢𝑇𝑤𝑗⁢𝑘⁢𝐼𝑗⁢𝑘
5−𝐻⁢𝑇+∑𝑗∈𝐺⁢𝐴⁢𝐵⁢𝐴𝑤𝑗⁢𝑘⁢𝐼𝑗⁢𝑘
𝐺⁢𝐴⁢𝐵⁢𝐴+𝐼𝑠⁢𝑤⁢𝑖⁢𝑚.
The currents 𝐼𝑣⁢𝑖⁢𝑠 and 𝐼𝑠⁢𝑤⁢𝑖⁢𝑚 are assumed to arise from glutamatergic input targeting the serotonergic and GABAergic neurons, respectively, consistent with glutamate imaging under the assumption that the first glutamate wave arises from spillover from glutamatergic axons not targeting serotonergic neurons, which did not show a response to the first glutamate wave.
Two-photon, plasma ablation of DRN GABAergic neurons
We performed two-photon plasma ablation of DRN GABAergic neurons and voltage imaging of serotonergic neurons before and after the ablation (Figures 6 and S8) by using triple transgenic zebrafish, Tg(tph2:Gal4; UAS:Voltron; gad1b:loxP-RFP-loxP-GFP), and an optical setup described in our previous work.98 Briefly, we first performed a volumetric scan of red fluorescence, which represents GABAergic neurons expressing RFP, using 561-nm CW laser (Omicron, Germany) and 630/92 nm filter (FF01-630/92, Semrock), to determine the site of ablation manually. Based on this acquired stack, we chose 40-70 GABAergic cells inside the DRN for the main ablation (Figure 6) or outside the DRN for control ablation (Figures S8C–S8F). The distribution of selected points is shown in Figures S8A and S8B. After the selection, we tuned the wavelength of a femtosecond laser (Camereon Ultra, Coherent) to 1050 nm and performed automatic ablation of selected cells. The lateral and depth positioning of the laser was controlled by a 2-axis galvanometer (Cambridge Technology) and a piezoelectric drive (Physik Instrument) of the objective lens. We chose the dwell time per cell to be 10-15 milliseconds because of the depth of this area. There are >100 GABAergic neurons in the larval zebrafish DRN,17 and we did not target all of them to avoid the risk of collateral damage to nearby serotonergic neurons.
We performed voltage imaging of serotonergic neurons that express Voltron conjugated with JF525 fluorescent dye for 5 minutes during the motor vigor learning task before and after the above ablation procedure.


# 9.5.1 Positron-2 analysis in whole brain paper

'''
Motion correction
We applied motion correction to each z-stack layer separately using a rigid motion correction method (NoRMCorre32).

ROI segmentation
We used manual ROI labeling for Fish 0 and Fish 1. During the manual annotation process, we labeled two types of ROIs. Firstly, we identified and outlined objects that were approximately the size of a neuron (∼9 pixels or 6.6 µm) and had ring-like boundaries. Secondly, for objects of a similar size to neurons, which we tentatively identified as neurons, we analyzed their temporal intensity traces. We then selected those objects whose intensity traces show narrow (less than 20ms), high amplitude (e.g., >2× the standard deviation of the traces), and positive-going spike signals. The annotated ROIs were reviewed by a second person. The percentage of ROIs that the second person disagreed with was ∼5% of all the annotated ROIs. The disagreement concentrated on the cases when examining the intensity traces of an object was needed to decide whether to label the object.

ROI temporal trace extraction
The extracted ROIs are processed by VolPy’s temporal trace extraction pipeline, which is composed of background removal, trace denoising and spike extraction. We used an adaptive threshold for spiking detection in VolPy. After VolPy’s pipeline, we further remove the ROIs with low SNRVolPy ( less than 4) or ROIs with large overlap. The overlap is determined by calculating the spatial correlation between each ROI and all other ROIs. ROIs with over 0.9 correlation between itself and any of existing ROIs were rejected as they are likely selecting the same cell.

Artifact removal
After extracting putative spiking activity from each Region of Interest (ROI), we employed the Uniform Manifold Approximation and Projection (UMAP) algorithm to identify and remove ROIs that may have been influenced by recording artifacts, such as those caused by excitation intensity variation due to blood flow. To achieve this, we first applied a 250-ms moving Hanning window to each ROI’s spike raster for smoothing and estimation of its firing rate. The entire recording data was structured into an N × D matrix, where N represented the number of ROIs and D represented the number of temporal samples. This matrix was fed to the UMAP algorithm and transformed into an N × 2 matrix, where each ROI was represented as a point within a 2-dimensional manifold. We then employed the Density-Based Spatial Clustering of Applications with Noise (DBSCAN) algorithm to identify clusters within the 2D representations of ROIs, effectively sorting ROIs with similar temporal spiking patterns into groups. We then visually identified the ROI cluster that was well-separated from the main cluster in the 2D UMAP representation. We then labeled the ROIs of the identified cluster with a unique cluster ID and removed them from the total set of ROIs. The remaining ROIs were then mapped and clustered again using UMAP-DBSCAN for the next iteration. The iteration was continued until the 2D UMAP representation became near Gaussian distributed. The remaining ROIs were assigned to a unique cluster. Following this clustering step, we scrutinized the temporal patterns and spatial distributions of ROIs within each group to detect any indications of artifacts resulting from recording imperfections, which were excluded from further analysis. The resulting ROIs were putatively treated as neurons, sorted into different groups using the same UMAP-DBSCAN algorithm, and await further analysis.
'''


## 9.5.7 Complete Pipeline Examples from Literature

### Positron2 Pipeline (Wang et al. - Whole-brain Zebrafish Voltage Imaging)

This pipeline was used for imaging Positron2 across entire larval zebrafish brains.

#### Manual ROI Annotation Criteria
```python
# ROI selection criteria from the paper:
ROI_CRITERIA = {
    'size_pixels': 9,           # ~6.6 µm diameter
    'shape': 'ring-like',       # Membrane-targeted indicator
    'spike_width_max_ms': 20,   # Narrow spikes
    'spike_amplitude_std': 2.0, # >2× std of trace
    'spike_polarity': 'positive',  # Note: Positron2 has POSITIVE spikes (unlike Voltron-2)
}
```

#### Overlap Removal (Spatial Correlation)
```python
def remove_overlapping_rois(roi_masks, traces, correlation_threshold=0.9):
    """
    Remove ROIs that are likely duplicates based on spatial correlation.
    From Wang et al.: ROIs with >0.9 correlation are rejected.
    """
    n_rois = len(roi_masks)
    keep_mask = np.ones(n_rois, dtype=bool)
    
    # Flatten masks for correlation
    masks_flat = roi_masks.reshape(n_rois, -1).astype(float)
    
    # Compute pairwise spatial correlations
    for i in range(n_rois):
        if not keep_mask[i]:
            continue
        for j in range(i + 1, n_rois):
            if not keep_mask[j]:
                continue
            
            # Spatial correlation between masks
            corr = np.corrcoef(masks_flat[i], masks_flat[j])[0, 1]
            
            if corr > correlation_threshold:
                # Keep the one with higher SNR
                snr_i = np.std(traces[i])  # Simplified SNR proxy
                snr_j = np.std(traces[j])
                if snr_i >= snr_j:
                    keep_mask[j] = False
                else:
                    keep_mask[i] = False
    
    return np.where(keep_mask)[0]
```

#### Iterative UMAP-DBSCAN Artifact Removal
```python
def iterative_umap_dbscan_artifact_removal(spike_rasters, fps=200,
                                            hanning_window_ms=250,
                                            max_iterations=10):
    """
    Iterative UMAP-DBSCAN clustering to remove artifact-contaminated ROIs.
    From Wang et al.: Iterate until 2D UMAP representation is near-Gaussian.
    
    Args:
        spike_rasters: list of binary spike trains per ROI
        fps: frame rate
        hanning_window_ms: smoothing window for firing rate estimation
        max_iterations: maximum clustering iterations
    
    Returns:
        clean_roi_indices: indices of ROIs that passed artifact removal
        artifact_roi_indices: indices of ROIs flagged as artifacts
        cluster_labels: final cluster assignments for clean ROIs
    """
    import umap
    from sklearn.cluster import DBSCAN
    from scipy.signal import convolve
    from scipy.stats import shapiro
    
    n_rois = len(spike_rasters)
    n_samples = len(spike_rasters[0])
    
    # Create smoothed firing rate matrix
    window_samples = int(hanning_window_ms * fps / 1000)
    hanning = np.hanning(window_samples)
    hanning /= hanning.sum()
    
    firing_rates = np.zeros((n_rois, n_samples))
    for i, raster in enumerate(spike_rasters):
        firing_rates[i] = convolve(raster.astype(float), hanning, mode='same')
    
    # Track which ROIs are still candidates
    candidate_mask = np.ones(n_rois, dtype=bool)
    artifact_indices = []
    
    for iteration in range(max_iterations):
        current_indices = np.where(candidate_mask)[0]
        if len(current_indices) < 10:
            break
        
        current_rates = firing_rates[current_indices]
        
        # UMAP embedding
        reducer = umap.UMAP(n_components=2, n_neighbors=min(15, len(current_indices)-1),
                           min_dist=0.1, random_state=42)
        embedding = reducer.fit_transform(current_rates)
        
        # Check if distribution is approximately Gaussian (stopping criterion)
        # Use Shapiro-Wilk test on each dimension
        _, p_x = shapiro(embedding[:, 0]) if len(embedding) >= 3 else (0, 1)
        _, p_y = shapiro(embedding[:, 1]) if len(embedding) >= 3 else (0, 1)
        
        if p_x > 0.05 and p_y > 0.05:
            # Distribution is approximately Gaussian - stop iterating
            print(f"Iteration {iteration}: UMAP distribution is Gaussian, stopping.")
            break
        
        # DBSCAN clustering
        clusterer = DBSCAN(eps=0.5, min_samples=3)
        labels = clusterer.fit_predict(embedding)
        
        # Find outlier cluster (well-separated from main cluster)
        unique_labels = set(labels) - {-1}  # Exclude noise
        if len(unique_labels) <= 1:
            break
        
        # Main cluster = largest cluster
        cluster_sizes = {l: np.sum(labels == l) for l in unique_labels}
        main_cluster = max(cluster_sizes, key=cluster_sizes.get)
        
        # Find clusters that are well-separated (potential artifacts)
        main_centroid = embedding[labels == main_cluster].mean(axis=0)
        
        for cluster_id in unique_labels:
            if cluster_id == main_cluster:
                continue
            
            cluster_centroid = embedding[labels == cluster_id].mean(axis=0)
            distance = np.linalg.norm(cluster_centroid - main_centroid)
            
            # If cluster is far from main cluster, flag as artifact
            main_std = embedding[labels == main_cluster].std()
            if distance > 3 * main_std:
                # Mark these ROIs as artifacts
                artifact_mask = labels == cluster_id
                artifact_roi_indices = current_indices[artifact_mask]
                artifact_indices.extend(artifact_roi_indices)
                candidate_mask[artifact_roi_indices] = False
                print(f"Iteration {iteration}: Removed {len(artifact_roi_indices)} artifact ROIs")
    
    clean_indices = np.where(candidate_mask)[0]
    return clean_indices, np.array(artifact_indices)
```

### SNR Calculation (VolPy-style)
```python
def compute_snr_volpy(trace, spike_frames, fps=200):
    """
    Compute SNR as defined in VolPy.
    SNR = median(spike_amplitudes) / noise_std
    
    Noise is estimated from spike-free regions.
    """
    if len(spike_frames) == 0:
        return 0
    
    # For Voltron-2: invert trace (negative spikes)
    # For Positron2: use trace directly (positive spikes)
    
    # Get spike amplitudes (peak - local baseline)
    window = int(10 * fps / 1000)  # 10ms window
    amplitudes = []
    
    for frame in spike_frames:
        if frame < window or frame >= len(trace) - window:
            continue
        
        # Local baseline (before spike)
        baseline = np.median(trace[frame-window:frame-2])
        # Peak value
        peak = trace[frame]
        amplitudes.append(np.abs(peak - baseline))
    
    if len(amplitudes) == 0:
        return 0
    
    # Estimate noise from spike-free regions
    spike_free_mask = np.ones(len(trace), dtype=bool)
    for frame in spike_frames:
        start = max(0, frame - window)
        end = min(len(trace), frame + window)
        spike_free_mask[start:end] = False
    
    if spike_free_mask.sum() < 100:
        noise_std = np.std(trace)
    else:
        noise_std = np.std(trace[spike_free_mask])
    
    snr = np.median(amplitudes) / (noise_std + 1e-10)
    return snr

def filter_by_snr(traces, spike_times_list, fps=200, min_snr=4):
    """
    Filter ROIs by SNR threshold.
    Wang et al. used SNR > 4 as cutoff.
    """
    keep_indices = []
    snr_values = []
    
    for i, (trace, spike_times) in enumerate(zip(traces, spike_times_list)):
        spike_frames = (np.array(spike_times) * fps).astype(int)
        snr = compute_snr_volpy(trace, spike_frames, fps)
        snr_values.append(snr)
        
        if snr >= min_snr:
            keep_indices.append(i)
    
    return keep_indices, snr_values
```

### Second Paper Pipeline (Camera Noise Correction, Local PCA, Semi-NMF, LSTM)

If you have the second paper's methods available, please share them and I'll add detailed implementations for:
- Camera noise correction
- dipy-based motion correction  
- Local PCA denoising specifics
- Semi-NMF segmentation details
- LSTM spike detection
- Subthreshold activity extraction

In the meantime, here are general implementations of the key techniques mentioned:

#### Camera Noise Correction
```python
def correct_camera_noise(video, dark_frame=None, flat_field=None):
    """
    Correct for camera-specific noise patterns.
    
    Args:
        video: raw video (n_frames, height, width)
        dark_frame: average of frames with no illumination (fixed pattern noise)
        flat_field: normalized response to uniform illumination
    
    Returns:
        corrected video
    """
    corrected = video.astype(np.float32)
    
    # Subtract dark frame (fixed pattern noise)
    if dark_frame is not None:
        corrected -= dark_frame
    
    # Divide by flat field (pixel sensitivity variation)
    if flat_field is not None:
        # Normalize flat field
        flat_norm = flat_field / np.mean(flat_field)
        flat_norm = np.clip(flat_norm, 0.1, 10)  # Avoid extreme corrections
        corrected /= flat_norm
    
    # Remove hot pixels (outliers in spatial domain)
    from scipy.ndimage import median_filter
    for i in range(len(corrected)):
        frame = corrected[i]
        median_frame = median_filter(frame, size=3)
        # Replace pixels that deviate significantly from local median
        outlier_mask = np.abs(frame - median_frame) > 5 * np.std(frame)
        corrected[i][outlier_mask] = median_frame[outlier_mask]
    
    return corrected

def estimate_dark_frame(dark_video):
    """Estimate dark frame from a recording with no illumination."""
    return np.median(dark_video, axis=0)

def estimate_flat_field(flat_video):
    """Estimate flat field from uniform illumination recording."""
    mean_frame = np.mean(flat_video, axis=0)
    # Normalize to mean of 1
    return mean_frame / np.mean(mean_frame)
```

#### Background Subtraction Methods
```python
def subtract_background_annulus(video, roi_mask, inner_gap=2, outer_width=5):
    """
    Subtract local background using annulus around ROI.
    Common in voltage imaging to remove neuropil contamination.
    """
    from scipy.ndimage import binary_dilation, binary_erosion
    
    # Create annulus mask
    dilated = binary_dilation(roi_mask, iterations=inner_gap + outer_width)
    inner = binary_dilation(roi_mask, iterations=inner_gap)
    annulus = dilated & ~inner
    
    # Extract traces
    roi_trace = np.mean(video[:, roi_mask], axis=1)
    bg_trace = np.mean(video[:, annulus], axis=1) if annulus.sum() > 0 else 0
    
    # Subtract with optional coefficient
    return roi_trace - 0.7 * bg_trace  # 0.7 is typical neuropil coefficient

def subtract_background_percentile(trace, percentile=8, window_sec=1.0, fps=200):
    """
    Rolling percentile baseline subtraction.
    """
    from scipy.ndimage import percentile_filter
    window = int(window_sec * fps)
    baseline = percentile_filter(trace, percentile, size=window)
    return trace - baseline
```

#### Baseline Correction and dF/F Computation
```python
def compute_dff(trace, method='percentile', percentile=10, window_sec=None, fps=200):
    """
    Compute ΔF/F with various baseline methods.
    
    Methods:
        'percentile': global percentile as F0
        'rolling_percentile': sliding window percentile
        'exponential': fit exponential decay (for photobleaching)
    """
    if method == 'percentile':
        f0 = np.percentile(trace, percentile)
        dff = (trace - f0) / (f0 + 1e-10)
        
    elif method == 'rolling_percentile':
        from scipy.ndimage import percentile_filter
        window = int((window_sec or 2.0) * fps)
        f0 = percentile_filter(trace, percentile, size=window)
        dff = (trace - f0) / (f0 + 1e-10)
        
    elif method == 'exponential':
        # Fit exponential decay for photobleaching correction
        from scipy.optimize import curve_fit
        
        def exp_decay(t, a, b, c):
            return a * np.exp(-b * t) + c
        
        t = np.arange(len(trace))
        try:
            # Use robust fitting (median of rolling windows as targets)
            window = fps  # 1 second windows
            n_windows = len(trace) // window
            t_fit = np.array([i * window + window//2 for i in range(n_windows)])
            y_fit = np.array([np.median(trace[i*window:(i+1)*window]) for i in range(n_windows)])
            
            popt, _ = curve_fit(exp_decay, t_fit, y_fit,
                               p0=[trace[0]-trace[-1], 0.001, trace[-1]],
                               maxfev=1000)
            f0 = exp_decay(t, *popt)
        except:
            f0 = np.percentile(trace, percentile)
        
        dff = (trace - f0) / (f0 + 1e-10)
    
    return dff
```

#### Subthreshold Activity Extraction
```python
def extract_subthreshold(trace, spike_frames, fps=200,
                         interpolation_window_ms=20,
                         lowpass_cutoff_hz=10):
    """
    Extract subthreshold membrane potential by removing spikes and lowpass filtering.
    
    1. Interpolate over spike regions
    2. Lowpass filter to get slow membrane potential fluctuations
    """
    from scipy.interpolate import interp1d
    from scipy.signal import butter, filtfilt
    
    # Create mask of spike regions to interpolate
    window = int(interpolation_window_ms * fps / 1000)
    spike_mask = np.zeros(len(trace), dtype=bool)
    for frame in spike_frames:
        start = max(0, frame - window//2)
        end = min(len(trace), frame + window//2)
        spike_mask[start:end] = True
    
    # Interpolate over spikes
    x = np.arange(len(trace))
    valid_x = x[~spike_mask]
    valid_y = trace[~spike_mask]
    
    if len(valid_x) < 10:
        interpolated = trace.copy()
    else:
        f = interp1d(valid_x, valid_y, kind='linear',
                     bounds_error=False, fill_value='extrapolate')
        interpolated = f(x)
    
    # Lowpass filter
    nyquist = fps / 2
    b, a = butter(4, lowpass_cutoff_hz / nyquist, btype='low')
    subthreshold = filtfilt(b, a, interpolated)
    
    return subthreshold
```

#### Complete Pipeline Function
```python
def voltage_imaging_pipeline_full(video, fps=200, config=None):
    """
    Full voltage imaging pipeline with all stages.
    
    Args:
        video: (n_frames, height, width) raw video
        fps: frame rate
        config: dict of parameters for each stage
    
    Returns:
        dict with roi_masks, traces, spike_times, subthreshold, metadata
    """
    if config is None:
        config = {}
    
    results = {'metadata': {'fps': fps, 'config': config}}
    
    # === Stage 0: Camera noise correction ===
    if config.get('camera_correction', False):
        video = correct_camera_noise(video)
    
    # === Stage 1: Motion correction ===
    mc_method = config.get('motion_correction', 'template')
    if mc_method == 'template':
        video, shifts = motion_correct_template(video, max_shift=50)
        results['metadata']['motion_shifts'] = shifts
    elif mc_method == 'none':
        pass
    
    # === Stage 2: Denoising ===
    denoise_method = config.get('denoising', 'local_pca')
    if denoise_method == 'local_pca':
        video = local_pca_denoise(video,
                                   patch_size=config.get('pca_patch_size', 32),
                                   n_components=config.get('pca_components', 10))
    elif denoise_method == 'temporal_median':
        video = temporal_median_filter(video, window=3)
    
    # === Stage 3: ROI Segmentation ===
    seg_method = config.get('segmentation', 'correlation')
    std_proj = np.std(video, axis=0)
    
    if seg_method == 'correlation':
        corr_img = compute_correlation_image(video)
        roi_masks = segment_from_correlation(corr_img, std_proj)
    elif seg_method == 'ring':
        roi_masks = detect_ring_rois(std_proj)
    elif seg_method == 'threshold':
        # Simple threshold segmentation
        thresh = np.mean(std_proj) + 1.5 * np.std(std_proj)
        binary = std_proj > thresh
        binary = morphology.remove_small_objects(binary, min_size=30)
        labeled = measure.label(binary)
        roi_masks = np.array([labeled == i for i in range(1, labeled.max()+1)])
    
    results['roi_masks'] = roi_masks
    results['metadata']['n_rois_initial'] = len(roi_masks)
    
    # === Stage 4: Trace extraction ===
    traces = []
    for mask in roi_masks:
        if config.get('background_subtraction', 'annulus') == 'annulus':
            trace = subtract_background_annulus(video, mask)
        else:
            trace = np.mean(video[:, mask], axis=1)
        
        # Compute dF/F
        dff = compute_dff(trace,
                         method=config.get('baseline_method', 'rolling_percentile'),
                         fps=fps)
        traces.append(dff)
    
    traces = np.array(traces)
    results['traces'] = traces
    
    # === Stage 5: Spike detection ===
    spike_method = config.get('spike_detection', 'adaptive')
    spike_times = []
    spike_frames = []
    
    for trace in traces:
        if spike_method == 'adaptive':
            times, _ = detect_spikes_adaptive(trace, fps)
        elif spike_method == 'mad':
            times = detect_spikes_mad(trace, fps)
        elif spike_method == 'template':
            times = detect_spikes_template(trace, fps)
        else:
            # Simple threshold
            inverted = -trace
            threshold = np.mean(inverted) + 3 * np.std(inverted)
            peaks, _ = signal.find_peaks(inverted, height=threshold)
            times = peaks / fps
        
        spike_times.append(times)
        spike_frames.append((times * fps).astype(int))
    
    results['spike_times'] = spike_times
    results['spike_frames'] = spike_frames
    
    # === Stage 6: Quality filtering ===
    # SNR filter
    if config.get('snr_filter', True):
        keep_idx, snr_values = filter_by_snr(traces, spike_times, fps,
                                              min_snr=config.get('min_snr', 4))
        results['metadata']['snr_values'] = snr_values
    else:
        keep_idx = list(range(len(roi_masks)))
    
    # Overlap removal
    if config.get('overlap_removal', True):
        keep_idx_overlap = remove_overlapping_rois(
            roi_masks[keep_idx],
            traces[keep_idx],
            correlation_threshold=0.9
        )
        keep_idx = [keep_idx[i] for i in keep_idx_overlap]
    
    results['metadata']['n_rois_final'] = len(keep_idx)
    results['keep_indices'] = keep_idx
    
    # === Stage 7: Artifact removal (optional) ===
    if config.get('artifact_removal', False) and len(keep_idx) > 10:
        # Create spike rasters
        n_frames = video.shape[0]
        spike_rasters = []
        for idx in keep_idx:
            raster = np.zeros(n_frames)
            raster[spike_frames[idx]] = 1
            spike_rasters.append(raster)
        
        clean_idx, artifact_idx = iterative_umap_dbscan_artifact_removal(spike_rasters, fps)
        keep_idx = [keep_idx[i] for i in clean_idx]
        results['metadata']['n_rois_after_artifact_removal'] = len(keep_idx)
    
    # === Stage 8: Subthreshold extraction (optional) ===
    if config.get('extract_subthreshold', False):
        subthreshold = []
        for idx in keep_idx:
            sub = extract_subthreshold(traces[idx], spike_frames[idx], fps)
            subthreshold.append(sub)
        results['subthreshold'] = np.array(subthreshold)
    
    # Filter results to kept ROIs
    results['roi_masks_filtered'] = roi_masks[keep_idx]
    results['traces_filtered'] = traces[keep_idx]
    results['spike_times_filtered'] = [spike_times[i] for i in keep_idx]
    
    return results
```

### Kawashima et al. Pipeline (Voltron in Zebrafish - Raphe Serotonin Study)

This pipeline was used for voltage imaging of Voltron in zebrafish raphe neurons. Notable for detailed camera noise correction, local PCA denoising with whiteness stopping criterion, semi-NMF segmentation, and LSTM spike detection.

#### Step 1: Camera Noise Correction (Huang/Liu method)
```python
def camera_noise_correction_kawashima(video, dark_frames, calibration_data=None):
    """
    Camera noise correction from Kawashima et al.
    
    Corrected pixel: s_i = (s_i^r - o_i) / g_i
    
    Where:
    - s_i^r: raw readout at pixel i
    - o_i: camera offset (mean of dark frames)
    - g_i: camera gain (fitted from variance vs intensity relationship)
    
    Args:
        video: raw video (n_frames, height, width)
        dark_frames: video acquired with no illumination (for offset estimation)
        calibration_data: dict with multiple illumination levels for gain estimation
            {'level_k': {'mean': D_ik, 'variance': v_ik}, ...}
    
    Returns:
        corrected video
    """
    # Estimate offset (o_i) from dark frames
    offset = np.mean(dark_frames, axis=0)  # Mean of ~60k dark images
    baseline_variance = np.var(dark_frames, axis=0)  # Variance in dark condition
    
    # Estimate gain (g_i) from calibration data at multiple illumination levels
    if calibration_data is not None:
        # Fit gain: minimize sum_k ((v_ik - v_i) - g_i * (D_ik - o_i))^2
        # This is linear regression: variance_excess = gain * intensity_excess
        
        h, w = offset.shape
        gain = np.ones((h, w), dtype=np.float32)
        
        levels = sorted(calibration_data.keys())
        for y in range(h):
            for x in range(w):
                # Collect (intensity, variance) pairs across illumination levels
                intensities = []
                variances = []
                for level in levels:
                    D_ik = calibration_data[level]['mean'][y, x]
                    v_ik = calibration_data[level]['variance'][y, x]
                    
                    intensity_excess = D_ik - offset[y, x]
                    variance_excess = v_ik - baseline_variance[y, x]
                    
                    if intensity_excess > 0:
                        intensities.append(intensity_excess)
                        variances.append(variance_excess)
                
                if len(intensities) >= 2:
                    # Linear regression through origin: variance = gain * intensity
                    intensities = np.array(intensities)
                    variances = np.array(variances)
                    gain[y, x] = np.sum(intensities * variances) / np.sum(intensities ** 2)
                    gain[y, x] = np.clip(gain[y, x], 0.1, 10)  # Reasonable bounds
    else:
        # Without calibration data, assume uniform gain
        gain = np.ones_like(offset)
    
    # Apply correction: s_i = (s_i^r - o_i) / g_i
    corrected = (video.astype(np.float32) - offset) / gain
    
    return corrected, {'offset': offset, 'gain': gain, 'baseline_variance': baseline_variance}

def estimate_camera_calibration(calibration_videos):
    """
    Estimate camera calibration from videos at multiple illumination levels.
    
    Args:
        calibration_videos: dict mapping illumination level to video array
            e.g., {0: dark_video, 5: video_5mW, 10: video_10mW, 18: video_18mW}
    
    Returns:
        calibration_data for camera_noise_correction_kawashima
    """
    calibration_data = {}
    for level, video in calibration_videos.items():
        calibration_data[level] = {
            'mean': np.mean(video, axis=0),
            'variance': np.var(video, axis=0)
        }
    return calibration_data
```

#### Step 2: Motion Correction (dipy-based)
```python
def motion_correct_dipy(video, reference='mean', subpixel=True):
    """
    2D rigid registration using dipy package.
    Subpixel-level correction for sequential images.
    
    Args:
        video: motion-corrected video (n_frames, height, width)
        reference: 'mean', 'first', or frame index
        subpixel: if True, use subpixel registration
    
    Returns:
        corrected video, shifts array
    """
    try:
        from dipy.align.imaffine import AffineRegistration, MutualInformationMetric
        from dipy.align.transforms import TranslationTransform2D
        USE_DIPY = True
    except ImportError:
        USE_DIPY = False
        print("dipy not installed, falling back to skimage")
    
    n_frames, h, w = video.shape
    
    # Create reference frame
    if reference == 'mean':
        ref_frame = np.mean(video, axis=0)
    elif reference == 'first':
        ref_frame = video[0]
    elif isinstance(reference, int):
        ref_frame = video[reference]
    else:
        ref_frame = np.mean(video, axis=0)
    
    corrected = np.zeros_like(video, dtype=np.float32)
    shifts = np.zeros((n_frames, 2))
    
    if USE_DIPY:
        # dipy-based registration
        metric = MutualInformationMetric(nbins=32)
        affreg = AffineRegistration(metric=metric)
        transform = TranslationTransform2D()
        
        for i in range(n_frames):
            frame = video[i].astype(np.float64)
            ref = ref_frame.astype(np.float64)
            
            # Register
            affine = affreg.optimize(ref, frame, transform, params0=None)
            
            # Extract translation
            shifts[i] = affine.affine[:2, 2]
            
            # Apply transformation
            from scipy.ndimage import affine_transform
            corrected[i] = affine_transform(frame, affine.affine[:2, :2],
                                            offset=affine.affine[:2, 2])
    else:
        # Fallback to skimage phase_cross_correlation
        from skimage.registration import phase_cross_correlation
        from scipy.ndimage import shift as ndi_shift
        
        for i in range(n_frames):
            frame = video[i].astype(np.float64)
            
            shift, error, diffphase = phase_cross_correlation(
                ref_frame, frame, upsample_factor=10 if subpixel else 1
            )
            shifts[i] = shift
            corrected[i] = ndi_shift(frame, shift, mode='constant', cval=0)
    
    return corrected, shifts
```

#### Step 3: Local PCA Denoising (with whiteness criterion)
```python
def local_pca_denoise_kawashima(video, patch_size=64, overlap=32, max_components=50):
    """
    Local PCA denoising with automatic rank selection.
    
    Stop adding components when residual is statistically white
    (within 99% confidence interval).
    
    From Kawashima et al.:
    - Find low-rank representation Y_bar = sum_k u_k * v_k
    - Stop when residual R_k = Y - Y_bar is white noise
    
    Args:
        video: (n_frames, height, width) motion-corrected video
        patch_size: size of spatial patches
        overlap: overlap between patches
        max_components: maximum number of PCA components
    
    Returns:
        denoised video
    """
    from scipy.stats import chi2
    
    n_frames, h, w = video.shape
    stride = patch_size - overlap
    
    # Output arrays
    denoised = np.zeros_like(video, dtype=np.float64)
    weights = np.zeros((h, w), dtype=np.float64)
    
    def is_white_noise(residual, confidence=0.99):
        """
        Test if residual is statistically white using Ljung-Box test.
        For simplicity, we check if autocorrelation at lag 1 is near zero.
        """
        # Flatten spatial dimensions
        n_frames, n_pixels = residual.shape
        
        # Sample some pixels for efficiency
        sample_idx = np.random.choice(n_pixels, min(100, n_pixels), replace=False)
        
        autocorrs = []
        for idx in sample_idx:
            trace = residual[:, idx]
            if np.std(trace) > 1e-10:
                # Lag-1 autocorrelation
                autocorr = np.corrcoef(trace[:-1], trace[1:])[0, 1]
                autocorrs.append(autocorr)
        
        if len(autocorrs) == 0:
            return True
        
        # For white noise, autocorrelation should be ~N(0, 1/n_frames)
        # Check if mean absolute autocorrelation is small
        mean_abs_autocorr = np.mean(np.abs(autocorrs))
        threshold = 2.58 / np.sqrt(n_frames)  # 99% CI for autocorr of white noise
        
        return mean_abs_autocorr < threshold
    
    # Process each patch
    for y_start in range(0, h - patch_size + 1, stride):
        for x_start in range(0, w - patch_size + 1, stride):
            # Extract patch: (n_frames, patch_size, patch_size)
            patch = video[:, y_start:y_start+patch_size, x_start:x_start+patch_size]
            patch_flat = patch.reshape(n_frames, -1)  # (n_frames, n_pixels)
            
            # Center the data
            mean_vals = patch_flat.mean(axis=0)
            centered = patch_flat - mean_vals
            
            # Iterative PCA with whiteness stopping criterion
            U, S, Vt = np.linalg.svd(centered, full_matrices=False)
            
            # Find optimal number of components
            best_k = 1
            for k in range(1, min(max_components, len(S))):
                # Reconstruct with k components
                reconstructed = U[:, :k] @ np.diag(S[:k]) @ Vt[:k, :]
                residual = centered - reconstructed
                
                if is_white_noise(residual):
                    best_k = k
                    break
                best_k = k
            
            # Final reconstruction with best_k components
            reconstructed = U[:, :best_k] @ np.diag(S[:best_k]) @ Vt[:best_k, :]
            reconstructed += mean_vals
            reconstructed = reconstructed.reshape(n_frames, patch_size, patch_size)
            
            # Add to output with overlap weighting
            denoised[:, y_start:y_start+patch_size, x_start:x_start+patch_size] += reconstructed
            weights[y_start:y_start+patch_size, x_start:x_start+patch_size] += 1
    
    # Handle edges (patches that don't fit)
    # ... (simplified: just use available data)
    
    # Normalize by overlap count
    weights = np.maximum(weights, 1)  # Avoid division by zero
    denoised /= weights[np.newaxis, :, :]
    
    return denoised.astype(np.float32)
```

#### Step 4: Semi-NMF Cell Segmentation
```python
def semi_nmf_segmentation_kawashima(video, n_components=100, correlation_threshold=0.8,
                                     max_iter=500, min_roi_size=20):
    """
    Semi-nonnegative matrix factorization for cell segmentation.
    
    From Kawashima et al.:
    minimize ||Y - A*F - B||^2
    subject to: A >= 0, B = b * 1^T, b >= 0
    
    Where:
    - A: spatial components (non-negative ROI masks)
    - F: temporal components (can be negative - voltage traces)
    - B: temporally constant background
    
    Initialization: super-pixels with local correlation > 0.8
    
    Args:
        video: denoised video (n_frames, height, width)
        n_components: number of components to extract
        correlation_threshold: for super-pixel initialization
        max_iter: maximum iterations for optimization
        min_roi_size: minimum ROI size in pixels
    
    Returns:
        roi_masks: (n_rois, height, width) boolean masks
        temporal_components: (n_rois, n_frames) fluorescence traces
        background: (height, width) static background
    """
    n_frames, h, w = video.shape
    n_pixels = h * w
    
    # Reshape video: Y is (n_pixels, n_frames)
    Y = video.reshape(n_frames, n_pixels).T.astype(np.float64)
    
    # === Step 1: Initialize with super-pixels ===
    def compute_local_correlation_map(video, radius=2):
        """Compute local correlation for each pixel."""
        corr_map = np.zeros((h, w))
        video_norm = (video - video.mean(axis=0)) / (video.std(axis=0) + 1e-10)
        
        for dy in range(-radius, radius+1):
            for dx in range(-radius, radius+1):
                if dy == 0 and dx == 0:
                    continue
                shifted = np.roll(np.roll(video_norm, dy, axis=1), dx, axis=2)
                corr = (video_norm * shifted).mean(axis=0)
                corr_map += corr
        
        corr_map /= (2*radius + 1)**2 - 1
        return corr_map
    
    def find_super_pixels(video, corr_threshold=0.8):
        """Find super-pixels: connected regions with high local correlation."""
        corr_map = compute_local_correlation_map(video)
        binary = corr_map > corr_threshold
        
        from scipy.ndimage import label
        labeled, n_labels = label(binary)
        
        super_pixels = []
        for i in range(1, n_labels + 1):
            mask = labeled == i
            if mask.sum() >= min_roi_size:
                super_pixels.append(mask)
        
        return super_pixels
    
    super_pixels = find_super_pixels(video, correlation_threshold)
    n_init = min(len(super_pixels), n_components)
    
    if n_init == 0:
        print("Warning: No super-pixels found, using random initialization")
        # Random initialization
        n_init = n_components
        super_pixels = []
        for _ in range(n_init):
            mask = np.zeros((h, w), dtype=bool)
            cy, cx = np.random.randint(10, h-10), np.random.randint(10, w-10)
            mask[cy-3:cy+3, cx-3:cx+3] = True
            super_pixels.append(mask)
    
    # Initialize A (spatial components)
    A = np.zeros((n_pixels, n_init))
    for i, mask in enumerate(super_pixels[:n_init]):
        A[:, i] = mask.flatten().astype(np.float64)
    
    # Initialize background b (mean of low-variance pixels)
    pixel_var = Y.var(axis=1)
    low_var_mask = pixel_var < np.percentile(pixel_var, 20)
    b = Y[low_var_mask].mean(axis=1) if low_var_mask.sum() > 0 else Y.mean(axis=1)
    b = np.maximum(b, 0)  # Non-negative background
    
    # B = b * 1^T
    B = np.outer(b, np.ones(n_frames)) if len(b) == n_pixels else np.zeros((n_pixels, n_frames))
    
    # === Step 2: Alternating optimization ===
    for iteration in range(max_iter):
        # Update F (temporal components): F = (A^T A)^-1 A^T (Y - B)
        Y_bg = Y - B
        AtA = A.T @ A + 1e-6 * np.eye(A.shape[1])  # Regularization
        AtY = A.T @ Y_bg
        F = np.linalg.solve(AtA, AtY)
        
        # Update A (spatial components): A = (Y - B) F^T (F F^T)^-1, then clip to >= 0
        FFt = F @ F.T + 1e-6 * np.eye(F.shape[0])
        YFt = Y_bg @ F.T
        A = np.linalg.solve(FFt.T, YFt.T).T
        A = np.maximum(A, 0)  # Non-negative constraint
        
        # Update background b: b = mean((Y - A*F), axis=1), clipped to >= 0
        residual = Y - A @ F
        b = residual.mean(axis=1)
        b = np.maximum(b, 0)
        B = np.outer(b, np.ones(n_frames))
        
        # Check convergence
        if iteration % 50 == 0:
            reconstruction_error = np.linalg.norm(Y - A @ F - B, 'fro')
            print(f"Iteration {iteration}: reconstruction error = {reconstruction_error:.2f}")
    
    # === Step 3: Extract ROI masks ===
    roi_masks = []
    temporal_components = []
    
    for i in range(A.shape[1]):
        spatial = A[:, i].reshape(h, w)
        temporal = F[i, :]
        
        # Threshold spatial component to get mask
        thresh = np.percentile(spatial[spatial > 0], 90) if (spatial > 0).sum() > 0 else 0
        mask = spatial > thresh
        
        # Clean up mask
        mask = morphology.remove_small_objects(mask, min_size=min_roi_size)
        
        if mask.sum() >= min_roi_size:
            roi_masks.append(mask)
            temporal_components.append(temporal)
    
    return (np.array(roi_masks),
            np.array(temporal_components),
            b.reshape(h, w))
```

#### Step 5: LSTM Spike Detection
```python
def create_lstm_spike_detector(window_size=41, hidden_size=64, dropout=0.3):
    """
    Create LSTM network for spike detection.
    
    From Kawashima et al.:
    - Two LSTM layers with dropout between them
    - Input: time series window (41 frames = 136.67 ms at 300 Hz)
    - Output: probability of spike at center of window
    - Trained on simultaneous ephys + imaging data
    
    Args:
        window_size: input window size (41 frames in paper)
        hidden_size: LSTM hidden layer size
        dropout: dropout rate between LSTM layers
    
    Returns:
        PyTorch model (or TensorFlow/Keras equivalent)
    """
    try:
        import torch
        import torch.nn as nn
        
        class LSTMSpikeDetector(nn.Module):
            def __init__(self, input_size=1, hidden_size=64, dropout=0.3):
                super().__init__()
                self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
                self.dropout = nn.Dropout(dropout)
                self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
                self.fc = nn.Linear(hidden_size, 1)
                self.sigmoid = nn.Sigmoid()
            
            def forward(self, x):
                # x: (batch, window_size, 1)
                out, _ = self.lstm1(x)
                out = self.dropout(out)
                out, _ = self.lstm2(out)
                # Take output at last time step
                out = self.fc(out[:, -1, :])
                return self.sigmoid(out)
        
        return LSTMSpikeDetector(hidden_size=hidden_size, dropout=dropout)
    
    except ImportError:
        # Keras/TensorFlow fallback
        try:
            from tensorflow import keras
            from tensorflow.keras import layers
            
            model = keras.Sequential([
                layers.LSTM(hidden_size, return_sequences=True,
                           input_shape=(window_size, 1)),
                layers.Dropout(dropout),
                layers.LSTM(hidden_size),
                layers.Dense(1, activation='sigmoid')
            ])
            model.compile(optimizer='adam', loss='binary_crossentropy',
                         metrics=['accuracy'])
            return model
        
        except ImportError:
            print("Neither PyTorch nor TensorFlow available")
            return None

def detect_spikes_lstm(trace, model, fps=300, window_size=41, threshold=0.5):
    """
    Detect spikes using trained LSTM model.
    
    Args:
        trace: ΔF/F trace (n_frames,)
        model: trained LSTM spike detector
        fps: frame rate (300 Hz in Kawashima paper)
        window_size: input window size
        threshold: probability threshold for spike detection
    
    Returns:
        spike_times: array of spike times in seconds
    """
    import torch
    
    n_frames = len(trace)
    half_window = window_size // 2
    
    # Normalize trace
    trace_norm = (trace - np.mean(trace)) / (np.std(trace) + 1e-10)
    
    # Slide window across trace
    spike_probs = np.zeros(n_frames)
    
    model.eval()
    with torch.no_grad():
        for i in range(half_window, n_frames - half_window):
            window = trace_norm[i - half_window:i + half_window + 1]
            window_tensor = torch.FloatTensor(window).unsqueeze(0).unsqueeze(-1)
            
            prob = model(window_tensor).item()
            spike_probs[i] = prob
    
    # Find peaks above threshold
    from scipy.signal import find_peaks
    peaks, _ = find_peaks(spike_probs, height=threshold, distance=int(0.002 * fps))
    
    spike_times = peaks / fps
    return spike_times, spike_probs

def train_lstm_spike_detector(traces, ground_truth_spikes, fps=300,
                               window_size=41, epochs=100, batch_size=64):
    """
    Train LSTM spike detector on paired imaging + electrophysiology data.
    
    Args:
        traces: list of ΔF/F traces
        ground_truth_spikes: list of spike frame indices from electrophysiology
        fps: frame rate
        window_size: input window size
        epochs: training epochs
        batch_size: training batch size
    
    Returns:
        trained model
    """
    import torch
    from torch.utils.data import DataLoader, TensorDataset
    
    # Create training data
    X = []
    y = []
    half_window = window_size // 2
    
    for trace, spikes in zip(traces, ground_truth_spikes):
        trace_norm = (trace - np.mean(trace)) / (np.std(trace) + 1e-10)
        spike_set = set(spikes)
        
        for i in range(half_window, len(trace) - half_window):
            window = trace_norm[i - half_window:i + half_window + 1]
            label = 1.0 if i in spike_set else 0.0
            X.append(window)
            y.append(label)
    
    X = np.array(X)
    y = np.array(y)
    
    # Balance classes (spike vs no-spike)
    spike_idx = np.where(y == 1)[0]
    no_spike_idx = np.where(y == 0)[0]
    n_samples = min(len(spike_idx), len(no_spike_idx))
    
    balanced_idx = np.concatenate([
        spike_idx[:n_samples],
        np.random.choice(no_spike_idx, n_samples, replace=False)
    ])
    np.random.shuffle(balanced_idx)
    
    X = X[balanced_idx]
    y = y[balanced_idx]
    
    # Create model and train
    model = create_lstm_spike_detector(window_size=window_size)
    
    X_tensor = torch.FloatTensor(X).unsqueeze(-1)
    y_tensor = torch.FloatTensor(y).unsqueeze(-1)
    
    dataset = TensorDataset(X_tensor, y_tensor)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.BCELoss()
    
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_X, batch_y in loader:
            optimizer.zero_grad()
            output = model(batch_X)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: loss = {total_loss/len(loader):.4f}")
    
    return model
```

#### Step 6: Subthreshold Activity Estimation
```python
def extract_subthreshold_kawashima(trace, spike_frames, fps=300,
                                    median_window_ms=70, clip_window_frames=2):
    """
    Estimate subthreshold activity using rolling median.
    
    From Kawashima et al.:
    - Rolling median filter with 70 ms window
    - Clip frames around spikes (-1 to +1 frames) to avoid spike nonlinearity
    
    Args:
        trace: ΔF/F trace
        spike_frames: detected spike frame indices
        fps: frame rate
        median_window_ms: median filter window (70 ms in paper)
        clip_window_frames: frames to clip around spikes (1 in paper)
    
    Returns:
        subthreshold: subthreshold membrane potential estimate
    """
    from scipy.ndimage import median_filter
    
    n_frames = len(trace)
    
    # Create mask of frames to exclude (around spikes)
    valid_mask = np.ones(n_frames, dtype=bool)
    for spike_frame in spike_frames:
        start = max(0, spike_frame - clip_window_frames)
        end = min(n_frames, spike_frame + clip_window_frames + 1)
        valid_mask[start:end] = False
    
    # Interpolate over clipped regions
    trace_interpolated = trace.copy()
    if not valid_mask.all():
        valid_indices = np.where(valid_mask)[0]
        invalid_indices = np.where(~valid_mask)[0]
        
        if len(valid_indices) > 2:
            from scipy.interpolate import interp1d
            f = interp1d(valid_indices, trace[valid_indices],
                        kind='linear', bounds_error=False,
                        fill_value='extrapolate')
            trace_interpolated[invalid_indices] = f(invalid_indices)
    
    # Apply rolling median filter
    window_samples = int(median_window_ms * fps / 1000)
    window_samples = window_samples if window_samples % 2 == 1 else window_samples + 1
    
    subthreshold = median_filter(trace_interpolated, size=window_samples)
    
    return subthreshold
```

#### ΔF/F Computation (Kawashima method)
```python
def compute_dff_kawashima(trace, fps=300, baseline_percentile=20,
                           baseline_window_sec=180):
    """
    Compute ΔF/F using running percentile baseline.
    
    From Kawashima et al.:
    ΔF/F = (F - F0) / F0
    where F0 is running 20th percentile within 3-minute window
    
    For Voltron (negative indicator): use -ΔF/F for analysis
    
    Args:
        trace: raw fluorescence trace
        fps: frame rate
        baseline_percentile: percentile for baseline (20 in paper)
        baseline_window_sec: window size in seconds (180 = 3 minutes)
    
    Returns:
        dff: ΔF/F trace
        dff_inverted: -ΔF/F (for Voltron analysis)
    """
    from scipy.ndimage import percentile_filter
    
    window_samples = int(baseline_window_sec * fps)
    
    # Running percentile baseline
    F0 = percentile_filter(trace, baseline_percentile, size=window_samples)
    
    # Compute ΔF/F
    dff = (trace - F0) / (F0 + 1e-10)
    
    # Inverted for Voltron (negative indicator)
    dff_inverted = -dff
    
    return dff, dff_inverted
```

#### Behavioral Kernel Fitting (GLM for spike prediction)
```python
def fit_spike_glm_kernels(spike_times, behavior_data, fps=300,
                          history_sec=1.0, l2_reg=0.01):
    """
    Fit GLM kernels to predict spikes from behavioral variables.
    
    From Kawashima et al.:
    P(spike at time t) = Binomial(w_s^T * S_t + w_v^T * V_t - w_sp^T * SP_t)
    
    Where:
    - S_t: swim vigor history
    - V_t: visual input history  
    - SP_t: recent spike history
    - w_s, w_v, w_sp: learned kernels
    
    Args:
        spike_times: array of spike times in seconds
        behavior_data: dict with 'swim_vigor', 'visual_input' arrays
        fps: frame rate
        history_sec: history window for kernels (1 second in paper)
        l2_reg: L2 regularization strength
    
    Returns:
        kernels: dict with fitted w_s, w_v, w_sp
        model: fitted sklearn LogisticRegression
        performance: explained variance on validation set
    """
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_score
    
    n_history = int(history_sec * fps)  # 300 time points for 1 sec at 300 Hz
    
    swim_vigor = behavior_data.get('swim_vigor', np.zeros(10000))
    visual_input = behavior_data.get('visual_input', np.zeros(10000))
    n_frames = len(swim_vigor)
    
    # Create spike raster
    spike_raster = np.zeros(n_frames)
    spike_frames = (np.array(spike_times) * fps).astype(int)
    spike_frames = spike_frames[(spike_frames >= 0) & (spike_frames < n_frames)]
    spike_raster[spike_frames] = 1
    
    # Build design matrix
    # Use sqrt of swim vigor (as in paper)
    swim_vigor_sqrt = np.sqrt(np.abs(swim_vigor)) * np.sign(swim_vigor)
    
    X = []
    y = []
    
    for t in range(n_history, n_frames):
        # Swim vigor history
        s_t = swim_vigor_sqrt[t-n_history:t]
        # Visual input history
        v_t = visual_input[t-n_history:t]
        # Spike history
        sp_t = spike_raster[t-n_history:t]
        
        features = np.concatenate([s_t, v_t, sp_t])
        X.append(features)
        y.append(spike_raster[t])
    
    X = np.array(X)
    y = np.array(y)
    
    # Balance classes (sample around spike events)
    spike_idx = np.where(y == 1)[0]
    no_spike_idx = np.where(y == 0)[0]
    
    # Sample to balance ~50/50
    n_samples = min(len(spike_idx) * 2, len(no_spike_idx))
    balanced_no_spike = np.random.choice(no_spike_idx, n_samples, replace=False)
    balanced_idx = np.concatenate([spike_idx, balanced_no_spike])
    np.random.shuffle(balanced_idx)
    
    X_balanced = X[balanced_idx]
    y_balanced = y[balanced_idx]
    
    # Fit logistic regression with L2 regularization
    model = LogisticRegression(penalty='l2', C=1/l2_reg, max_iter=1000)
    model.fit(X_balanced, y_balanced)
    
    # Extract kernels
    coef = model.coef_[0]
    kernels = {
        'swim_vigor': coef[:n_history],
        'visual_input': coef[n_history:2*n_history],
        'spike_history': coef[2*n_history:]
    }
    
    # Cross-validation performance
    cv_scores = cross_val_score(model, X_balanced, y_balanced, cv=5, scoring='roc_auc')
    
    return kernels, model, {'cv_auc': cv_scores.mean(), 'cv_std': cv_scores.std()}

def plot_behavioral_kernels(kernels, fps=300):
    """Plot fitted behavioral kernels."""
    fig, axes = plt.subplots(1, 3, figsize=(12, 3))
    
    time_ms = np.arange(len(kernels['swim_vigor'])) * 1000 / fps - len(kernels['swim_vigor']) * 1000 / fps
    
    axes[0].plot(time_ms, kernels['swim_vigor'])
    axes[0].set_xlabel('Time before spike (ms)')
    axes[0].set_ylabel('Weight')
    axes[0].set_title('Swim Vigor Kernel')
    axes[0].axhline(0, color='gray', linestyle='--', alpha=0.5)
    
    axes[1].plot(time_ms, kernels['visual_input'])
    axes[1].set_xlabel('Time before spike (ms)')
    axes[1].set_title('Visual Input Kernel')
    axes[1].axhline(0, color='gray', linestyle='--', alpha=0.5)
    
    axes[2].plot(time_ms, -kernels['spike_history'])  # Negative because it's subtracted
    axes[2].set_xlabel('Time before spike (ms)')
    axes[2].set_title('Spike History Kernel (refractory)')
    axes[2].axhline(0, color='gray', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    return fig
```

#### Population Coding Analysis (sparse LDA)
```python
def compute_population_coding(neural_activity, conditions, l2_gamma=0.5):
    """
    Compute population coding direction using sparse linear discriminant analysis.
    
    From Kawashima et al.:
    l = argmin_l -(l^T (r_high - r_low))^2 / (l^T Σ_r l)
    
    With regularized covariance: Σ_r = (1-γ) * cov(r) + γ * I
    
    Args:
        neural_activity: (n_neurons, n_timepoints) firing rates
        conditions: array of condition labels (e.g., 'high' or 'low')
        l2_gamma: regularization parameter [0, 1]
    
    Returns:
        coding_direction: (n_neurons,) vector
        explained_variance: fraction of variance explained
    """
    unique_conditions = np.unique(conditions)
    if len(unique_conditions) != 2:
        raise ValueError("Exactly 2 conditions required")
    
    cond1, cond2 = unique_conditions
    
    # Mean activity per condition
    r_cond1 = neural_activity[:, conditions == cond1].mean(axis=1)
    r_cond2 = neural_activity[:, conditions == cond2].mean(axis=1)
    
    # Regularized covariance
    r_centered = neural_activity - neural_activity.mean(axis=1, keepdims=True)
    cov_r = np.cov(r_centered)
    n_neurons = cov_r.shape[0]
    
    sigma_r = (1 - l2_gamma) * cov_r + l2_gamma * np.eye(n_neurons)
    
    # Solve for coding direction (Fisher LDA)
    # l = Σ_r^-1 (r_high - r_low)
    diff = r_cond2 - r_cond1
    coding_direction = np.linalg.solve(sigma_r, diff)
    
    # Normalize
    coding_direction /= np.linalg.norm(coding_direction)
    
    # Compute explained variance
    projected = coding_direction @ neural_activity
    proj_cond1 = projected[conditions == cond1]
    proj_cond2 = projected[conditions == cond2]
    
    between_var = (proj_cond1.mean() - proj_cond2.mean())**2
    within_var = proj_cond1.var() + proj_cond2.var()
    explained_variance = between_var / (between_var + within_var + 1e-10)
    
    return coding_direction, explained_variance
```